In [64]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
%matplotlib inline

### Initial Exploration of Data

In [49]:
national_covid_df = pd.read_csv('nation_level_daily.csv')

In [50]:
national_covid_df.head()

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
0,30 January,1,1,0,0,0,0
1,31 January,0,1,0,0,0,0
2,01 February,0,1,0,0,0,0
3,02 February,1,2,0,0,0,0
4,03 February,1,3,0,0,0,0


In [51]:
fig = px.scatter(x=national_covid_df.Date, y=national_covid_df["Daily Confirmed"])
fig.show()

We can see that the trend line is steeping up! We will try to map nifty 50 index and see how it works. 

In [52]:
nifty50_df = pd.read_csv('data.csv')

In [53]:
nifty50_df.head()

,Date,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr)
0,30-Jan-2020,12147.75,12150.30,12010.60,12035.80,538138743,19372.67
1,31-Jan-2020,12100.40,12103.55,11945.85,11962.10,771278258,26044.91
2,01-Feb-2020,11939.00,12017.35,11633.30,11661.85,537634767,20598.12
3,03-Feb-2020,11627.45,11749.85,11614.50,11707.90,669815788,25415.26
4,04-Feb-2020,11786.25,11986.15,11783.40,11979.65,560430291,22338.50


In [54]:
fig = px.scatter(x=nifty50_df.Date, y=nifty50_df.Close)
fig.show()

we will now superimpose these two plots to see if covid and nifty50 has any realtion.

In order to superimpose the two graphs we need to make sure that we take the same dates from both the data frames. There were days when stock market was closed but unfortunately covid-19 has not taken any pause.

### Data Cleaning

In [55]:
national_covid_df.Date

0       30 January 
1       31 January 
2      01 February 
3      02 February 
4      03 February 
           ...     
139        17 June 
140        18 June 
141        19 June 
142        20 June 
143        21 June 
Name: Date, Length: 144, dtype: object

In [56]:
nifty50_df.Date

0     30-Jan-2020
1     31-Jan-2020
2     01-Feb-2020
3     03-Feb-2020
4     04-Feb-2020
         ...     
90    15-Jun-2020
91    16-Jun-2020
92    17-Jun-2020
93    18-Jun-2020
94    19-Jun-2020
Name: Date, Length: 95, dtype: object

We need to modify `national_covid_df` to match with `nifty50_df` Date column. Then we can do an inner join.

In [57]:
def func_to_modify_date(x):
    x = x.split(" ")
    x[1] = x[1][0:3]
    x[2] = "2020"
    return "-".join(x)

national_covid_df.Date = national_covid_df.Date.apply(lambda x : func_to_modify_date(x))

In [58]:
national_covid_df.Date

0      30-Jan-2020
1      31-Jan-2020
2      01-Feb-2020
3      02-Feb-2020
4      03-Feb-2020
          ...     
139    17-Jun-2020
140    18-Jun-2020
141    19-Jun-2020
142    20-Jun-2020
143    21-Jun-2020
Name: Date, Length: 144, dtype: object

Great! Now we have both the data frames in the same data format.

In [59]:
merged_df = pd.merge(national_covid_df, nifty50_df, how='inner' ,on="Date")
merged_df

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr)
0,30-Jan-2020,1,1,0,0,0,0,12147.75,12150.30,12010.60,12035.80,538138743,19372.67
1,31-Jan-2020,0,1,0,0,0,0,12100.40,12103.55,11945.85,11962.10,771278258,26044.91
2,01-Feb-2020,0,1,0,0,0,0,11939.00,12017.35,11633.30,11661.85,537634767,20598.12
3,03-Feb-2020,1,3,0,0,0,0,11627.45,11749.85,11614.50,11707.90,669815788,25415.26
4,04-Feb-2020,0,3,0,0,0,0,11786.25,11986.15,11783.40,11979.65,560430291,22338.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,15-Jun-2020,10032,343075,10639,180324,396,9918,9919.35,9943.35,9726.35,9813.70,716053036,31926.65
91,16-Jun-2020,11086,354161,7226,187550,2004,11922,10014.80,10046.15,9728.50,9914.00,899895044,36257.89
92,17-Jun-2020,13108,367269,6890,194440,341,12263,9876.70,10003.60,9833.80,9881.15,685418390,30700.69
93,18-Jun-2020,13829,381098,10741,205181,343,12606,9863.25,10111.20,9845.05,10091.65,622895963,28594.32


Great now the shape of the above df matches the length of `nifty50_df`

In [71]:
fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=merged_df.Date, y=merged_df.Close),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=merged_df.Date, y=merged_df["Daily Confirmed"]),
    row=3, col=1
)

fig.update_layout(height=600, width=600, title_text="Nifty50 vs Daily Confirmed India")
fig.show()


This is really interesting, it seems that Indian markets were not reacting with daily confirmed cases in India. It will be interesting to see if markets reacted with US/world wide cases.

In [79]:
world_covid_df = pd.read_csv("world-covid-data.csv")

In [81]:
world_covid_df

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
0,AFG,Asia,Afghanistan,2019-12-31,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
1,AFG,Asia,Afghanistan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
2,AFG,Asia,Afghanistan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
3,AFG,Asia,Afghanistan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
4,AFG,Asia,Afghanistan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25919,NaN,NaN,International,2020-02-28,705.0,0.0,4.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25920,NaN,NaN,International,2020-02-29,705.0,0.0,6.0,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25921,NaN,NaN,International,2020-03-01,705.0,0.0,6.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25922,NaN,NaN,International,2020-03-02,705.0,0.0,6.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will first take US numbers.

In [85]:
US_covid = world_covid_df[world_covid_df.location=="United States"]
US_covid

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
24501,USA,North America,United States,2019-12-31,0.0,0.0,0.0,0.0,0.000,0.000,...,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86
24502,USA,North America,United States,2020-01-01,0.0,0.0,0.0,0.0,0.000,0.000,...,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86
24503,USA,North America,United States,2020-01-02,0.0,0.0,0.0,0.0,0.000,0.000,...,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86
24504,USA,North America,United States,2020-01-03,0.0,0.0,0.0,0.0,0.000,0.000,...,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86
24505,USA,North America,United States,2020-01-04,0.0,0.0,0.0,0.0,0.000,0.000,...,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24673,USA,North America,United States,2020-06-20,2220961.0,29909.0,119112.0,678.0,6709.798,90.359,...,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86
24674,USA,North America,United States,2020-06-21,2255119.0,34158.0,119719.0,607.0,6812.994,103.196,...,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86
24675,USA,North America,United States,2020-06-22,2280912.0,25793.0,119975.0,256.0,6890.918,77.924,...,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86
24676,USA,North America,United States,2020-06-23,2312302.0,31390.0,120402.0,427.0,6985.751,94.833,...,9.732,54225.446,1.2,151.089,10.79,19.1,24.6,NaN,2.77,78.86


In [93]:
fig = make_subplots(rows=5, cols=1)

fig.add_trace(
    go.Scatter(x=merged_df.Date, y=merged_df.Close),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=US_covid.date, y=US_covid["total_cases"]),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=US_covid.date, y=US_covid["new_cases"]),
    row=5, col=1
)

fig.update_layout(height=600, width=600, title_text="Nifty50, Total Cases in US and Daily Confirmed US")
fig.show()

Here we can see, From March 13th the Nifty 50 started to decline rapidly. This correlated to the fact that cases in US started almost doubling after every two days. On March 24th, US crossed 50k total confirmed cases mark. Daily confirmed cases started to increase rapidly from March 12, 2020.

We know however that Italy and China were most affected regions. How did Indian Markets performed then?

In [94]:
Italy_covid = world_covid_df[world_covid_df.location=="Italy"]
Italy_covid

,iso_code,continent,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,...,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
11930,ITA,Europe,Italy,2019-12-31,0.0,0.0,0.0,0.0,0.000,0.000,...,16.24,35220.084,2.0,113.151,4.78,19.8,27.8,NaN,3.18,83.51
11931,ITA,Europe,Italy,2020-01-01,0.0,0.0,0.0,0.0,0.000,0.000,...,16.24,35220.084,2.0,113.151,4.78,19.8,27.8,NaN,3.18,83.51
11932,ITA,Europe,Italy,2020-01-02,0.0,0.0,0.0,0.0,0.000,0.000,...,16.24,35220.084,2.0,113.151,4.78,19.8,27.8,NaN,3.18,83.51
11933,ITA,Europe,Italy,2020-01-03,0.0,0.0,0.0,0.0,0.000,0.000,...,16.24,35220.084,2.0,113.151,4.78,19.8,27.8,NaN,3.18,83.51
11934,ITA,Europe,Italy,2020-01-04,0.0,0.0,0.0,0.0,0.000,0.000,...,16.24,35220.084,2.0,113.151,4.78,19.8,27.8,NaN,3.18,83.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12102,ITA,Europe,Italy,2020-06-20,238011.0,-148.0,34561.0,47.0,3936.550,-2.448,...,16.24,35220.084,2.0,113.151,4.78,19.8,27.8,NaN,3.18,83.51
12103,ITA,Europe,Italy,2020-06-21,238275.0,264.0,34610.0,49.0,3940.916,4.366,...,16.24,35220.084,2.0,113.151,4.78,19.8,27.8,NaN,3.18,83.51
12104,ITA,Europe,Italy,2020-06-22,238499.0,224.0,34634.0,24.0,3944.621,3.705,...,16.24,35220.084,2.0,113.151,4.78,19.8,27.8,NaN,3.18,83.51
12105,ITA,Europe,Italy,2020-06-23,238720.0,221.0,34657.0,23.0,3948.276,3.655,...,16.24,35220.084,2.0,113.151,4.78,19.8,27.8,NaN,3.18,83.51


In [95]:
fig = make_subplots(rows=5, cols=1)

fig.add_trace(
    go.Scatter(x=merged_df.Date, y=merged_df.Close),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=Italy_covid.date, y=Italy_covid["total_cases"]),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=Italy_covid.date, y=Italy_covid["new_cases"]),
    row=5, col=1
)

fig.update_layout(height=600, width=600, title_text="Nifty50, Total Cases in US and Daily Confirmed US")
fig.show()

Let's import oil prices as well and see if that makes any sense - 

In [72]:
crude_oil_prices_df = pd.read_csv("brent-daily_csv.csv")

In [73]:
crude_oil_prices_df.head()

,Date,Price
0,1987-05-20,18.63
1,1987-05-21,18.45
2,1987-05-22,18.55
3,1987-05-25,18.60
4,1987-05-26,18.63


Ok so the date is again in another format and the Prices are in dollars. Also We don't need that historical data. Anyways, just for fun let us plot this data.

In [74]:
covid_crude_oil_price_df = crude_oil_prices_df[8301:]
#covid_crude_oil_price_df
fig = px.scatter(x=covid_crude_oil_price_df.Date, y=covid_crude_oil_price_df.Price)
fig.show()

In [75]:
def date_modify(x):
    x = x.split("-")
    dict_month = {"01":"Jan", "02":"Feb", "03":"Mar", "04":"Apr", "05":"May", "06":"Jun"}
    return x[2]+"-"+dict_month[x[1]]+"-"+x[0]

In [76]:
covid_crude_oil_price_df.Date = covid_crude_oil_price_df.Date.apply(lambda x: date_modify(x))
covid_crude_oil_price_df

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Price
8301,30-Jan-2020,57.72
8302,31-Jan-2020,57.77
8303,03-Feb-2020,54.00
8304,04-Feb-2020,53.90
8305,05-Feb-2020,55.36
...,...,...
8390,09-Jun-2020,40.45
8391,10-Jun-2020,41.18
8392,11-Jun-2020,37.76
8393,12-Jun-2020,38.54


In [77]:
merge_financial_markets = pd.merge(covid_crude_oil_price_df, merged_df, how="inner", on="Date")
merge_financial_markets

,Date,Price,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr)
0,30-Jan-2020,57.72,1,1,0,0,0,0,12147.75,12150.30,12010.60,12035.80,538138743,19372.67
1,31-Jan-2020,57.77,0,1,0,0,0,0,12100.40,12103.55,11945.85,11962.10,771278258,26044.91
2,03-Feb-2020,54.00,1,3,0,0,0,0,11627.45,11749.85,11614.50,11707.90,669815788,25415.26
3,04-Feb-2020,53.90,0,3,0,0,0,0,11786.25,11986.15,11783.40,11979.65,560430291,22338.50
4,05-Feb-2020,55.36,0,3,0,0,0,0,12005.85,12098.15,11953.35,12089.15,758032580,22998.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,09-Jun-2020,40.45,9981,276002,5634,134653,272,7749,10181.15,10291.15,10021.45,10046.65,754217460,36201.05
84,10-Jun-2020,41.18,11156,287158,6275,140928,358,8107,10072.60,10148.75,10036.85,10116.15,632383068,28485.47
85,11-Jun-2020,37.76,11135,298293,6044,146972,394,8501,10094.10,10112.05,9885.05,9902.00,715453486,30813.85
86,12-Jun-2020,38.54,11306,309599,7263,154235,388,8889,9544.95,9996.05,9544.35,9972.90,796306399,36505.77


We will now import the vix index and superimpose it with nifty50 close prices. Let's see what story they say.